# Manipulating models

In [1]:
from cobra.io import read_sbml_model
model = read_sbml_model('data/e_coli_core.xml.gz')

## Making temporary changes to the model

Usually one relies on making copies if objects need to be changed but the original state needs to be retained. Unfortunately, making copies of models is time consuming.

In [2]:
model.optimize()

,fluxes,reduced_costs
ACALD,0.0000,6.9389e-18
ACALDt,0.0000,0.0000e+00
ACKr,0.0000,1.0408e-17
ACONTa,6.0072,0.0000e+00
ACONTb,6.0072,1.3878e-17
...,...,...
TALA,1.4970,-1.3878e-17
THD2,0.0000,-2.5462e-03
TKT1,1.4970,-1.3878e-17
TKT2,1.1815,1.3878e-17


In [3]:
%%time
copy_of_model = model.copy()

CPU times: user 41.2 ms, sys: 6.45 ms, total: 47.7 ms
Wall time: 83 ms


Yes, even milliseconds add up pretty quickly if you need to run many simulation (e.g. if you need to knock out every single gene individually in the model to check if it is essential or not).

In [4]:
%%time
for gene in model.genes:
    mutant = model.copy()
    mutant.genes.get_by_id(gene.id).knock_out()

CPU times: user 3.58 s, sys: 251 ms, total: 3.83 s
Wall time: 5.4 s


For that reason cobrapy provides a mechanism that is less time consuming. Almost all methods that make changes to the mdoel such as knocking-out genes, reactions, adding or removing metabolites, reactions etc can be automatically reverted upon exit from a python context. How this works is probably best understood by looking at an example.

In [5]:
%%time
with model:
    for gene in model.genes:
        gene.knock_out()

CPU times: user 23.8 ms, sys: 1.89 ms, total: 25.7 ms
Wall time: 61.3 ms


Here, the `with model` statements starts the context and changes done to the model one indentation level to the right, are automatically recorded. When that block finishes, the context manager is requested to roll-back all changes leaving the model looking exactly as it did before all the changes.

Changing flux bounds can as indicated also be done reversibly. For example let's set the lower and upper bound of phosphoglycerate kinase to 0 (effectively knocking out the reaction).

In [6]:
with model:
    model.reactions.PGK.bounds = 0, 0
    print("PGK's bounds inside the with statement")
    print(model.reactions.PGK.lower_bound, model.reactions.PGK.bounds)
    print('Mutant growth rate: ', model.optimize().objective_value)
print("PGK's bounds outside the with statement")
print(model.reactions.PGK.bounds)

PGK's bounds inside the with statement
0 (0, 0)
Mutant growth rate:  -2.171174082614637e-15
PGK's bounds outside the with statement
(-1000.0, 1000.0)


## Slim versus full optimize

Mathematical solvers are now so fast that for many small to mid-size models computing the solution can be even faster than it takes us to collect the values from the solver and convert that to objects that are usable for in python. When we use `model.optimize` we gather values for all reactions and metabolites and that can take some time. If we are only interested in the flux value of a single reaction or the objective, it is faster to instead use `model.slim_optimize` which only does the optimization and returns the objective value leaving it up to you to fetch other values that you may need. For example, let's optimize and get the flux value of the `ATPM` reaction.

In [7]:
%%time
solution = model.optimize()
solution.fluxes['ATPM']

CPU times: user 5.37 ms, sys: 1.49 ms, total: 6.86 ms
Wall time: 24.9 ms


In [8]:
%%time
model.slim_optimize()
model.reactions.ATPM.flux

CPU times: user 358 µs, sys: 21 µs, total: 379 µs
Wall time: 388 µs


Again, the difference may seem small but when done thousands of times these small differences can start to become significant.

## Changing the medium

One can access the medium condition using `model.medium`. The indicated bound is the effective upper uptake bound. 

In [9]:
model.medium

{'EX_co2_e': 1000.0,
 'EX_glc__D_e': 10.0,
 'EX_h2o_e': 1000.0,
 'EX_h_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_pi_e': 1000.0}

Changing the carbon source in the medium can be achieved by adjusting the flux bounds of the respective exchange reactions appropriately. For example, the following code block removes glucose from the medium and adds succinate.

In [10]:
medium = model.medium
with model:
    medium['EX_glc__D_e'] = 0
    medium['EX_succ_e'] = 10
    model.medium = medium
    solution = model.optimize()
    print(solution.fluxes['BIOMASS_Ecoli_core_w_GAM'])

0.397563015428


Changing the carbon source to succinate led to a significant drop in growth rate.

## Exercise (10 min)

* Change the carbon source in the medium to a different carbon source. What is the difference in the growth rate observed?
* How about growing E. coli under anaerobic conditions?

## Adding reactions and pathways

In [11]:
from cobra import Reaction, Metabolite

Ok, let's create a new reactions.

In [12]:
new_reaction = Reaction('alchemy')

This reaction is going to convert water into gold (unfortunately lead is not part of _E. coli _ metabolism; creating wine would be blasphemy). So we need to create a new metabolite, since gold is not yet part of _E. coli's_ native metabolism.

In [13]:
gold = Metabolite(id='gold_c', compartment='c')

Now, we're going to specify the reaction's stoichiometry.

In [14]:
new_reaction.add_metabolites({model.metabolites.h2o_c: -1, gold: 1})

Printing the reaction reveals that the reaction indeed converts water into gold.

In [15]:
print(new_reaction.build_reaction_string())

h2o_c --> gold_c


Now, let's add the new reaction to the model.

In [16]:
model.add_reactions([new_reaction])

Quickly check that the reaction was indeed added to the model.

In [17]:
model.reactions.alchemy

Reaction identifier,alchemy
Name,
Memory address,0x0115d19908
Stoichiometry,h2o_c --> gold_c H2O -->
GPR,
Lower bound,0.0
Upper bound,1000.0


Let's produce some gold then!

In [18]:
model.objective = model.reactions.alchemy
model.optimize().objective_value

0.0

:-(

What happened? Forgot to add an exchange reaction so that gold can leave the system.

In [19]:
model.add_boundary(model.metabolites.gold_c, type='demand')

Reaction identifier,DM_gold_c
Name,demand
Memory address,0x0115d19cc0
Stoichiometry,gold_c --> -->
GPR,
Lower bound,0
Upper bound,1000.0


In [20]:
model.objective = model.reactions.alchemy
model.optimize().objective_value

1000.0

Yes, much better!

## Exercise (20 min)

* Add a pathway to the model (ideally one that you're personally interested in; you can also use a different model if you like). 